In [1]:
import pandas as pd
import json


# Defina o caminho para o seu arquivo CSV
input_csv_path = './data/data.csv'
# Defina o caminho de saída para o arquivo JSON processado
output_json_path = './data/top_products.json'

In [2]:

def process_sales_data(csv_path, num_top_products=20):
    """
    Processa o CSV de vendas para identificar os principais produtos.

    Args:
        csv_path (str): Caminho para o arquivo CSV de entrada.
        num_top_products (int): Número de produtos principais a serem extraídos.

    Returns:
        list: Uma lista de dicionários contendo os dados dos principais produtos.
    """
    print(f"Iniciando o carregamento de dados de: {csv_path}")
    try:
        # Carrega o CSV. Use 'sep=;' pois seus dados são separados por ponto e vírgula.
        # chunksize é usado para ler o arquivo em pedaços, economizando memória.
        chunks = pd.read_csv(csv_path, sep=';', chunksize=100000, encoding='utf-8')

        all_data = pd.DataFrame()
        for i, chunk in enumerate(chunks):
            print(f"Processando chunk {i+1}...")
            all_data = pd.concat([all_data, chunk], ignore_index=True)

        print("Dados carregados com sucesso. Iniciando o processamento...")

        # Garante que as colunas numéricas são do tipo correto
        all_data['TotalItem'] = pd.to_numeric(all_data['TotalItem'], errors='coerce')
        all_data['Controle'] = pd.to_numeric(all_data['Controle'], errors='coerce') # Para contar a incidência de vendas

        # Agrupar por produto para calcular o total de vendas (TotalItem)
        # e a contagem de vendas únicas (Controle)
        product_summary = all_data.groupby('ProNom').agg(
            total_vendas=('TotalItem', 'sum'),
            incidencia_vendas=('Controle', lambda x: x.nunique()) # Contar vendas únicas
        ).reset_index()

        # Calcular a porcentagem de incidência em vendas
        total_unique_sales = all_data['Controle'].nunique()
        product_summary['percent_incidencia'] = (product_summary['incidencia_vendas'] / total_unique_sales) * 100

        # Ordenar por total_vendas para pegar os principais produtos
        top_products = product_summary.sort_values(by='total_vendas', ascending=False).head(num_top_products)

        # Converter para o formato de lista de dicionários
        output_data = top_products.to_dict(orient='records')

        print("Processamento concluído.")
        return output_data

    except FileNotFoundError:
        print(f"Erro: O arquivo '{csv_path}' não foi encontrado.")
        return []
    except Exception as e:
        print(f"Ocorreu um erro durante o processamento: {e}")
        return []

if __name__ == "__main__":
    processed_data = process_sales_data(input_csv_path)

    if processed_data:
        with open(output_json_path, 'w', encoding='utf-8') as f:
            json.dump(processed_data, f, ensure_ascii=False, indent=4)
        print(f"Dados processados exportados para: {output_json_path}")
    else:
        print("Nenhum dado processado para exportar.")

Iniciando o carregamento de dados de: ./data/data.csv
Processando chunk 1...
Processando chunk 2...
Processando chunk 3...
Processando chunk 4...
Processando chunk 5...
Processando chunk 6...
Processando chunk 7...
Processando chunk 8...
Processando chunk 9...
Processando chunk 10...
Processando chunk 11...
Processando chunk 12...
Processando chunk 13...
Processando chunk 14...
Processando chunk 15...
Processando chunk 16...
Processando chunk 17...
Processando chunk 18...
Processando chunk 19...
Processando chunk 20...
Processando chunk 21...
Processando chunk 22...
Processando chunk 23...
Processando chunk 24...
Processando chunk 25...
Processando chunk 26...
Dados carregados com sucesso. Iniciando o processamento...
Processamento concluído.
Dados processados exportados para: ./data/top_products.json
